<a href="https://colab.research.google.com/github/imsazulfiqar/VoiceQuery_LLM_Response/blob/main/VoiceQuery_LLM_Response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faster-whisper
!pip install pydub
!pip install transformers
!pip install edge-tts
!pip install flash_attn
!pip install triton
!pip install gTTs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash_attn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.3 MB/s eta 0:00:00


In [ ]:
from faster_whisper import WhisperModel
from pydub import AudioSegment
from IPython.display import Audio, display, Javascript
from google.colab import output
import io
import base64
from pydub import AudioSegment
import numpy as np
import edge_tts
from transformers import AutoTokenizer, AutoModelForCausalLM
import soundfile as sf
from gtts import gTTS


In [ ]:
# Choosing the larger model for beter prediction
model_size = "large-v3"

# Running the model on CPU because of limited resources
model_Speech_to_Text = WhisperModel(model_size, device="cpu", compute_type="int8")


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

# Step 1

In [ ]:
# This code was mainly taken from the Source: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
# Edited the below for adjustments with the Faster whisper Speech to text Model


RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
# Making a function to record Audio

def record_audio(duration=10, sample_rate=16000):
    display(Javascript(RECORD))
    print(f"Recording for {duration} seconds...")
    audio_string = output.eval_js(f'record({duration * 1000})')
    audio_data = base64.b64decode(audio_string.split(',')[1])

    # Converting the audio data to an AudioSegment
    audio = AudioSegment.from_file(io.BytesIO(audio_data), format="webm")

    # Ensuring the audio is in 1 channel
    if audio.channels > 1:
        audio = audio.set_channels(1)

    # Setting the frame rate to the desired sample rate
    audio = audio.set_frame_rate(sample_rate)

    # Converting AudioSegment to NumPy array for compatibility with the Whisper model
    samples = np.array(audio.get_array_of_samples(), dtype=np.float32)

    # Normalizing the audio to range [-1, 1]
    samples = samples / np.abs(samples).max()

    print(f"Recording completed and returned as NumPy array.")
    return samples




In [ ]:
# Source: https://github.com/SYSTRAN/faster-whisper?tab=readme-ov-file
# Faster-Whisper model was selected because it is up to 4 times faster than
# openai/whisper for the same accuracy while using less memory.

def speechToText(audio):
    # Transcribing the audio
    segments, info = model_Speech_to_Text.transcribe(audio, vad_filter=True, vad_parameters={"threshold": 0.5}, language="en")

  # Checking if any segments were transcribed
    if not segments:
        print("No transcriptions found.")
        return ""

    # Converting segments to a list to make sure the transcription runs
    segments = list(segments)

    # Printing each segment's start and end time along with the transcribed text
    for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

    # Returning the full transcription as a concatenated string
    full_transcription = " ".join([segment.text for segment in segments])
    return full_transcription


In [ ]:
# Recording the audio
recorded_audio  = record_audio()

print("Recorded audio shape:", recorded_audio.shape)
print("Recorded audio content:", recorded_audio)
Audio(recorded_audio, rate=16000)



<IPython.core.display.Javascript object>

Recording for 10 seconds...
Recording completed and returned as NumPy array.
Recorded audio shape: (158400,)
Recorded audio content: [0.         0.         0.         ... 0.00665061 0.0042169  0.00188503]


In [ ]:
# Sending the recorded audio to the model for Transcriptions
full_transcription=speechToText(recorded_audio)

[0.18s -> 9.90s]  Hello, how are you?


# Step 2

In [ ]:
# Loading the tokenizer and model for GPT-2
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", device_map="cpu")

# Setting the pad token to the EOS token
tokenizer.pad_token = tokenizer.eos_token

def generate_response(transcribed_text, max_length=50):
    # Encoding the input text with padding and attention mask
    inputs = tokenizer(transcribed_text, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

    # Ensuring the pad token is set correctly
    inputs["input_ids"] = inputs["input_ids"].masked_fill(inputs["input_ids"] == tokenizer.pad_token_id, tokenizer.eos_token_id)
    inputs["attention_mask"] = inputs["input_ids"] != tokenizer.eos_token_id

    # Generating the response with attention mask, temperature, and repetition penalty
    outputs = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask, max_length=max_length, pad_token_id=tokenizer.eos_token_id,
                             temperature=0.7, repetition_penalty=1.2)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Printing input and response to verify the texts
    print("Transcribed Input:", transcribed_text)
    print("Generated Response:", response)

    # Skipping the first sentence if it repeats the inputted text
    sentences = response.split('. ')
    if sentences[0].strip().lower() == transcribed_text.strip().lower():
        sentences = sentences[1:]

    # Limiting the response to 2 sentences by joining only the first two sentences
    limited_response = '. '.join(sentences[:2])

    return limited_response



In [ ]:
#  Generating the response with restriction to 2 sentences
response = generate_response(full_transcription)
# print("LLM Response:", response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Transcribed Input:  Hello, how are you?
Generated Response:  Hello, how are you?
I'm a little bit of an idiot. I've been doing this for years and it's just not working out so well anymore! It was like my life had changed forever when the first time we met in high


# Step 3

In [ ]:
# Converting the generated response back to speech using Google TTS
def text_to_speech(text):

    # Initializing the gTTS object with the text
    tts = gTTS(text=text)

    # Saving the output as an mp3 file
    audio_file = "output_audio.mp3"
    tts.save(audio_file)

    print(f"Audio saved as '{audio_file}'")
    return audio_file

audio_file = text_to_speech(response)

# Playing the generated audio
Audio(audio_file)


Audio saved as 'output_audio.mp3'
